In [ ]:
from bokeh.plotting import figure, output_notebook, show

from shapely.geometry import LineString, Point

output_notebook()

In [ ]:
def draw(title, segment, recording, projected=None):
    p = figure(title=title)

    x, y = segment.coords.xy
    p.line(x, y, line_width=3, line_color="#ffe38c", legend="segment")
    p.circle(x, y, radius=0.05, line_width=3, line_color="#ffe38c", fill_color="white", legend="segment")

    x, y = recording.coords.xy
    p.line(x, y, line_width=3, line_color="#8cd8ff", legend="recording")
    p.circle(x, y, radius=0.05, line_width=3, line_color="#8cd8ff", fill_color="white", legend="recording")

    if projected is not None:
        x, y = projected.coords.xy
        p.line(x, y, line_width=1, line_color="#aa0011", legend="projected")
        p.circle(x, y, radius=0.05, line_width=1, line_color="#aa0011", fill_color="white", legend="projected")
    
    p.legend.background_fill_alpha = 0.7
    show(p)

In [ ]:
segment = LineString([(1,2), (3,-2), (5.7, 3.8), (9,10), (6,3)])
recording = LineString([(1.1, 2.2), (2.9, -1.8), (9.5, 10), (6.3, 3.5)])

draw("segment and recording", segment, recording)

In [ ]:
def bad_path_projection(recording, segment):
    projected = []
    for p in segment.coords:
        point_proj = recording.interpolate(recording.project(Point(p)))
        projected.append(point_proj)

    return LineString(projected)


projected = bad_path_projection(recording, segment)
draw("wrong segment projection", segment, recording, projected)

In [ ]:
from itertools import tee


def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


def correct_path_projection(recording, segment):
    projected = []
    
    recording = pairwise(recording.coords)
    
    # this is the first couple of points of the recording
    sec = LineString(next(recording))
    
    previous_distance = -1
    for point in segment.coords:
        while True:            
            distance = sec.project(Point(point), normalized=True)
            if distance >= 1.0 or distance < previous_distance:
                # move to the next couple of points
                try: 
                    sec = LineString(next(recording))
                    
                    # we must reset the previous distance
                    # because it only makes sense for the 
                    # previous couple of points
                    previous_distance = -1
                    
                    continue
                except StopIteration:
                    # its the last couple
                    pass

            projected.append(sec.interpolate(distance, normalized=True))
            previous_distance = distance
            
            # stop the inner loop to move to next point in the segment
            break
    
    return LineString(projected)

In [ ]:
projected = correct_path_projection(recording, segment)
draw("correct segment projection", segment, recording, projected)